We utilizerd the datasets from the pygod.

See [data repository](https://github.com/pygod-team/data) for more details.


In [2]:
import torch
from pygod.generator import gen_contextual_outliers, gen_structural_outliers
from pygod.utils import load_data
from pygod.models import MLPAE,SCAN,Radar,ANOMALOUS,GCNAE,DOMINANT,DONE,AdONE,AnomalyDAE,GAAN,GUIDE,CONAD
from pygod.metrics import eval_roc_auc
import numpy as np

dataset_name = "enron"
data = load_data(dataset_name)


contextual_n = 3
contextual_k = 25
structural_n = 3
structural_m = 25

data, yc = gen_contextual_outliers(data, n=contextual_n, k=contextual_k)
data, ys = gen_structural_outliers(data, n=structural_n, m=structural_m)
data.y = torch.logical_or(ys, yc).int()


/home/roy206/.conda/envs/cent7/2020.11-py38/NWRGAE/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Training and Testing
Training different model for different type of outliers



In [ ]:
# model = MLPAE(epoch=100,hid_dim=64)
# model = SCAN()
# model = Radar()
# model = ANOMALOUS(epoch=100)
# model = GCNAE(epoch=100,hid_dim=64)
# model = DOMINANT(epoch=50,hid_dim=64)
model = DONE(epoch=50,hid_dim=64)
# model = AdONE(epoch=100,hid_dim=64)
# model = AnomalyDAE(epoch=100)
# model = GAAN(epoch=100,hid_dim=64)
# model = GUIDE(epoch=10)
# model = CONAD(epoch=25,hid_dim=64)



roc_auc = []
roc_auc_c = []
roc_auc_s = []
for i in range(5):
    model.fit(data)
    labels = model.predict(data)
    outlier_scores = model.decision_function(data)
    prob = model.predict_proba(data)
    labels, confidence = model.predict(data, return_confidence=True)
    
    auc_score = eval_roc_auc(data.y.bool().numpy(), outlier_scores)
    auc_score = auc_score * 100
    
    auc_score_c = eval_roc_auc(yc, outlier_scores)
    auc_score_c = auc_score_c * 100
    
    auc_score_s = eval_roc_auc(ys, outlier_scores)
    auc_score_s = auc_score_s * 100
    
    roc_auc.append(auc_score)
    roc_auc_c.append(auc_score_c)
    roc_auc_s.append(auc_score_s)



roc_auc = np.array(roc_auc)
mean = np.mean(roc_auc)
std = np.std(roc_auc)

roc_auc_c = np.array(roc_auc_c)
mean_c = np.mean(roc_auc_c)
std_c = np.std(roc_auc_c)

roc_auc_s = np.array(roc_auc_s)
mean_s = np.mean(roc_auc_s)
std_s = np.std(roc_auc_s)

print(model)
print('Dataset: ', dataset_name)
print('%.2f ± %.2f' %(mean, std))
print('%.2f ± %.2f' %(mean_c, std_c))
print('%.2f ± %.2f' %(mean_s, std_s))